In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/Davidson_1.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,text,class
0,0,! ! ! mayasolovely : woman complain cleaning ...,0
1,1,! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...,0
2,2,! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...,0
3,3,! ! ! ! ! ! ! ! ! c g anderson : viva based l...,0
4,4,! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...,0


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

35736


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001, 0.001, 0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
17842/17842 [==============================] - 89s 5ms/step - loss: 0.2170 - acc: 0.9392
Epoch 2/3
17842/17842 [==============================] - 88s 5ms/step - loss: 0.1665 - acc: 0.9442
Epoch 3/3
17842/17842 [==============================] - 88s 5ms/step - loss: 0.1576 - acc: 0.9457
Epoch 1/3
17842/17842 [==============================] - 88s 5ms/step - loss: 0.2210 - acc: 0.9393
Epoch 2/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1675 - acc: 0.9436
Epoch 3/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1582 - acc: 0.9465
Epoch 1/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.2120 - acc: 0.9425
Epoch 2/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1656 - acc: 0.9446
Epoch 3/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1580 - acc: 0.9451
Epoch 1/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1838 - acc: 0.9405
Epoch 2/3


/home/bhargav/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Epoch 1/3
17842/17842 [==============================] - 83s 5ms/step - loss: 0.9283 - acc: 0.9410
Epoch 2/3
17842/17842 [==============================] - 83s 5ms/step - loss: 0.9296 - acc: 0.9423
Epoch 3/3
17842/17842 [==============================] - 83s 5ms/step - loss: 0.9296 - acc: 0.9423
Epoch 1/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1894 - acc: 0.9401
Epoch 2/3
17842/17842 [==============================] - 81s 5ms/step - loss: 0.1683 - acc: 0.9418
Epoch 3/3
17842/17842 [==============================] - 81s 5ms/step - loss: 0.1683 - acc: 0.9422


In [8]:
print(score)

[[0.6440967354344259, 0.5929184998952441, 0.6212375859434682], [0.6872121632704561, 0.6749278783110412, 0.6650641477376003], [0.48515057113187954, 0.48515057113187954, 0.5165378231976618]]


In [10]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(75, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Davidson_1.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Davidson_1.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Davidson_1.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Davidson_1.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
17842/17842 [==============================] - 99s 6ms/step - loss: 0.1841 - acc: 0.9409
Epoch 2/5
17842/17842 [==============================] - 90s 5ms/step - loss: 0.1539 - acc: 0.9446
Epoch 3/5
17842/17842 [==============================] - 91s 5ms/step - loss: 0.1404 - acc: 0.9482
Epoch 4/5
17842/17842 [==============================] - 82s 5ms/step - loss: 0.1226 - acc: 0.9540
Epoch 5/5
17842/17842 [==============================] - 86s 5ms/step - loss: 0.0941 - acc: 0.9640
Saved model to disk
Fold Number: 1
Epoch 1/5
17842/17842 [==============================] - 100s 6ms/step - loss: 0.1853 - acc: 0.9419
Epoch 2/5
17842/17842 [==============================] - 83s 5ms/step - loss: 0.1570 - acc: 0.9463
Epoch 3/5
17842/17842 [==============================] - 86s 5ms/step - loss: 0.1428 - acc: 0.9498
Epoch 4/5
17842/17842 [==============================] - 83s 5ms/step - loss: 0.1265 - acc: 0.9544
Epoch 5/5
17842/17842 [==============================] - 8

In [11]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.9424025100851636, 0.9430748543254146, 0.9428251121076233, 0.9385650224215246, 0.9338565022421524]
Mean: 0.9401448002363757
Std Dev: 0.0035469686229682496


In [12]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.6811016253685076, 0.6841144709747693, 0.6762956357547452, 0.673885777295294, 0.6435359220802092]
Mean: 0.6717866862947051
Std Dev: 0.014571100521454593


In [5]:
# load json and create model
json_file = open('Models/model_Davidson_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_Davidson_1.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f6c8c56db00> False
<keras.layers.recurrent.LSTM object at 0x7f6c8c56dc18> True
<keras.layers.core.Dense object at 0x7f6c8c5e9358> True
<keras.layers.core.Dense object at 0x7f6c8c56ddd8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f6c8c56db00> False
<keras.layers.recurrent.LSTM object at 0x7f6c8c56dc18> False
<keras.layers.core.Dense object at 0x7f6c8c5e9358> True
<keras.layers.core.Dense object at 0x7f6c8c56ddd8> True
Epoch 1/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.8993 - acc: 0.6334
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.3263 - acc: 0.8500
Epoch 3/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2893 - acc: 0.8731
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2671 - acc: 0.8823
Epoch 5/8
4460/4460 [==============================] - 8s 2ms/step - loss: 

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8859255939040789, 0.8865979381443299, 0.8811659192825112, 0.8959641255605382, 0.8943946188340807]
Mean: 0.8888096391451079
Std Dev: 0.005550035416135913
Macro-F1-Scores: 
[0.7630773804711521, 0.7730678971421359, 0.7604591142885053, 0.7982369522454602, 0.7982338328742675]
Mean: 0.7786150354043042
Std Dev: 0.016563476506899807


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.21402958314657106, 0.2124607799193187, 0.2116591928251121, 0.21143497757847535, 0.20919282511210763]
Mean: 0.21175547171631695
Std Dev: 0.0015715079505864398
Macro-F1-Scores: 
[0.20247825361250596, 0.20048922745058367, 0.19939535085089538, 0.19912246587334664, 0.19666319244264607]
Mean: 0.19962969804599556
Std Dev: 0.001895543098553781


In [10]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1


In [11]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f6c11878860> False
<keras.layers.recurrent.LSTM object at 0x7f6c118786d8> True
<keras.layers.core.Dense object at 0x7f6c1187f198> True
<keras.layers.core.Dense object at 0x7f6c842f1b38> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f6c11878860> False
<keras.layers.recurrent.LSTM object at 0x7f6c118786d8> False
<keras.layers.core.Dense object at 0x7f6c1187f198> True
<keras.layers.core.Dense object at 0x7f6c842f1b38> True
Epoch 1/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.6714 - acc: 0.7107
Epoch 2/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5463 - acc: 0.7466
Epoch 3/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5236 - acc: 0.7550
Epoch 4/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5064 - acc: 0.7642
Epoch 5/8
2841/2841 [==============================] - 5s 2ms/step - loss: 

In [12]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7611677805135421, 0.7575650950035187, 0.7565095003518649, 0.7610837438423645, 0.7754311862020415]
Mean: 0.7623514611826664
Std Dev: 0.006798900224497148
Macro-F1-Scores: 
[0.7063885384274331, 0.6875945984796598, 0.6758018146331171, 0.6855351938355891, 0.7082876801396698]
Mean: 0.6927215651030937
Std Dev: 0.012596013432023809


In [13]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6971508969398522, 0.6998592540464461, 0.7002111189303307, 0.7009148486981, 0.6972896867300247]
Mean: 0.6990851610689507
Std Dev: 0.001560760307448389
Macro-F1-Scores: 
[0.463581731796454, 0.4828519761255187, 0.47218247688303533, 0.4793355013138928, 0.4731710220127685]
Mean: 0.4742245416263339
Std Dev: 0.006619726097544439


In [14]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0


In [15]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f6c84253630> False
<keras.layers.recurrent.LSTM object at 0x7f6c118782e8> True
<keras.layers.core.Dense object at 0x7f6c86ec9d68> True
<keras.layers.core.Dense object at 0x7f6c8425fc88> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f6c84253630> False
<keras.layers.recurrent.LSTM object at 0x7f6c118782e8> False
<keras.layers.core.Dense object at 0x7f6c86ec9d68> True
<keras.layers.core.Dense object at 0x7f6c8425fc88> True
Epoch 1/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.8031 - acc: 0.6651
Epoch 2/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.6023 - acc: 0.6844
Epoch 3/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5678 - acc: 0.7075
Epoch 4/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5480 - acc: 0.7281
Epoch 5/8
2383/2383 [==============================] - 4s 2ms/step - loss: 

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7453859060402684, 0.7310113302559799, 0.7331095258078053, 0.7305916911456147, 0.7419219471254721]
Mean: 0.7364040800750281
Std Dev: 0.006080127596580997
Macro-F1-Scores: 
[0.6740962350832648, 0.6540442319717764, 0.658302265557786, 0.6567964745660507, 0.6800389050265425]
Mean: 0.664655622441084
Std Dev: 0.010397148713604863


In [17]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6715604026845637, 0.6689047419219472, 0.6697440201426773, 0.6647083508182963, 0.668485102811582]
Mean: 0.6686805236758133
Std Dev: 0.0022490597229128852
Macro-F1-Scores: 
[0.4183240488768441, 0.4149625255422805, 0.41984004791200125, 0.41326200914674455, 0.41593137254901963]
Mean: 0.41646400080537804
Std Dev: 0.002351524794741793


In [18]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0


In [19]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f6bee94a2e8> False
<keras.layers.recurrent.LSTM object at 0x7f6bee9a5588> True
<keras.layers.core.Dense object at 0x7f6bee94a5c0> True
<keras.layers.core.Dense object at 0x7f6bee9a5390> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f6bee94a2e8> False
<keras.layers.recurrent.LSTM object at 0x7f6bee9a5588> False
<keras.layers.core.Dense object at 0x7f6bee94a5c0> True
<keras.layers.core.Dense object at 0x7f6bee9a5390> True
Epoch 1/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.5166 - acc: 0.8195
Epoch 2/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4437 - acc: 0.8201
Epoch 3/8
1618/1618 [==============================] - 2s 2ms/step - loss: 0.4326 - acc: 0.8257
Epoch 4/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4240 - acc: 0.8294
Epoch 5/8
1618/1618 [==============================] - 2s 2ms/step - loss: 

In [20]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8295243977764052, 0.8318912237330037, 0.8238566131025958, 0.8275648949320148, 0.8281829419035847]
Mean: 0.8282040142895208
Std Dev: 0.0026324034915343494
Macro-F1-Scores: 
[0.5160632131624775, 0.5230449106988035, 0.49308269436886804, 0.500883969470713, 0.509754683337911]
Mean: 0.5085658942077546
Std Dev: 0.010639608776354146


In [21]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8245830759728228, 0.8244746600741656, 0.8195302843016069, 0.8238566131025958, 0.8250927070457355]
Mean: 0.8235074680993855
Std Dev: 0.0020270800835232566
Macro-F1-Scores: 
[0.4992070891458346, 0.48446363179435625, 0.47304646545762785, 0.5016550977334401, 0.49664000879434966]
Mean: 0.49100245858512165
Std Dev: 0.01074942417823275


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link


In [23]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f6bd38e39b0> False
<keras.layers.recurrent.LSTM object at 0x7f6bd38e3f60> True
<keras.layers.core.Dense object at 0x7f6bd38e3b00> True
<keras.layers.core.Dense object at 0x7f6bed380ac8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f6bd38e39b0> False
<keras.layers.recurrent.LSTM object at 0x7f6bd38e3f60> False
<keras.layers.core.Dense object at 0x7f6bd38e3b00> True
<keras.layers.core.Dense object at 0x7f6bed380ac8> True
Epoch 1/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3701 - acc: 0.8676
Epoch 2/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3192 - acc: 0.8837
Epoch 3/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.2975 - acc: 0.8884
Epoch 4/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.2843 - acc: 0.8915
Epoch 5/8
1926/1926 [==============================] - 3s 2ms/step - loss: 

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8884854771784232, 0.8883696780893042, 0.8883696780893042, 0.8904465212876428, 0.8920041536863966]
Mean: 0.8895351016662142
Std Dev: 0.0014659494805277415
Macro-F1-Scores: 
[0.5242047026279392, 0.5452294060603132, 0.5008961426692878, 0.5061388424730366, 0.5070483540715073]
Mean: 0.5167034895804169
Std Dev: 0.01627698826423369


In [25]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8547717842323651, 0.8613707165109035, 0.8582554517133957, 0.8644859813084113, 0.8561786085150571]
Mean: 0.8590125084560265
Std Dev: 0.003524426649088249
Macro-F1-Scores: 
[0.5435606060606061, 0.546304163126593, 0.5513006612750858, 0.5778965108106678, 0.5496182110581611]
Mean: 0.5537360304662228
Std Dev: 0.012372454762391658


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f1248f4b470> False
<keras.layers.recurrent.LSTM object at 0x7f1248f4bb38> True
<keras.layers.core.Dense object at 0x7f1248f4ba20> True
<keras.layers.core.Dense object at 0x7f1256b3dd68> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f1248f4b470> False
<keras.layers.recurrent.LSTM object at 0x7f1248f4bb38> False
<keras.layers.core.Dense object at 0x7f1248f4ba20> True
<keras.layers.core.Dense object at 0x7f1256b3dd68> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8483802006248973, 0.8506578947368421, 0.8501398256292153, 0.844546800460602, 0.8427372923178155]
Mean: 0.8472924027538744
Std Dev: 0.0031274596900408348
Macro-F1-Scores: 
[0.8438569210481112, 0.8455278328813868, 0.8453114339875141, 0.8397287361035841, 0.8385849910186799]
Mean: 0.8426019830078552
Std Dev: 0.0028937299111182644
Without Fine Tuning: 
Accuracies: 
[0.6740667653346489, 0.6774671052631579, 0.6739595328178977, 0.6691890113505511, 0.668695509129791]
Mean: 0.6726755847792093
Std Dev: 0.0033027192099995954
Macro-F1-Scores: 
[0.6051179678316014, 0.6120292765140374, 0.6049330503817354, 0.5962349538109305, 0.5967987487597463]
Mean: 0.6030227994596102
Std Dev: 0.005898628208137424


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f11de868b70> False
<keras.layers.recurrent.LSTM object at 0x7f11de868e48> True
<keras.layers.core.Dense object at 0x7f11de868da0> True
<keras.layers.core.Dense object at 0x7f11de85abe0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f11de868b70> False
<keras.layers.recurrent.LSTM object at 0x7f11de868e48> False
<keras.layers.core.Dense object at 0x7f11de868da0> True
<keras.layers.core.Dense object at 0x7f11de85abe0> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8581736750435431, 0.844449975111996, 0.8466898954703833, 0.8506719761075162, 0.8499253359880538]
Mean: 0.8499821715442986
Std Dev: 0.004669562491422108
Macro-F1-Scores: 
[0.7726479674351359, 0.7478450732014914, 0.7449120261200798, 0.770998109319051, 0.7643844749672835]
Mean: 0.7601575302086083
Std Dev: 0.011622549049308633
Without Fine Tuning: 
Accuracies: 
[0.7753172430952974, 0.772523643603783, 0.7755102040816326, 0.7732702837232454, 0.7777501244400199]
Mean: 0.7748742997887956
Std Dev: 0.0018422482191981556
Macro-F1-Scores: 
[0.521167403795953, 0.5208541107275284, 0.5307047257960926, 0.522063572995872, 0.5293565657799173]
Mean: 0.5248292758190727
Std Dev: 0.004286668243450305


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson_1.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_1_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_1_model.load_weights("Models/model_Davidson_1.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_1_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_1_model.layers:
        print(layer, layer.trainable)
    Davidson_1_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_1_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_1_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fd0a43d60b8> False
<keras.layers.recurrent.LSTM object at 0x7fd0a43d6208> True
<keras.layers.core.Dense object at 0x7fd0a4389be0> True
<keras.layers.core.Dense object at 0x7fd0a4392198> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fd0a43d60b8> False
<keras.layers.recurrent.LSTM object at 0x7fd0a43d6208> False
<keras.layers.core.Dense object at 0x7fd0a4389be0> True
<keras.layers.core.Dense object at 0x7fd0a4392198> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.9494047619047619, 0.9493545183714002, 0.9353876739562624, 0.9463220675944334, 0.9532803180914513]
Mean: 0.9467498679836618
Std Dev: 0.0060952753879420654
Macro-F1-Scores: 
[0.9319047928121725, 0.9316974883795162, 0.9145258925501976, 0.927754489743808, 0.9372047466316047]
Mean: 0.9286174820234597
Std Dev: 0.007659655532501788
Without Fine Tuning: 
Accuracies: 
[0.9027777777777778, 0.9036742800397219, 0.8946322067594433, 0.8966202783300199, 0.8876739562624254]
Mean: 0.8970756998338777
Std Dev: 0.005841674467917582
Macro-F1-Scores: 
[0.8531459798890388, 0.8548588172536713, 0.8388142182559117, 0.8424185463659148, 0.8259768689675231]
Mean: 0.8430428861464119
Std Dev: 0.010499610523667952
